In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chains.summarize import load_summarize_chain

In [15]:
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama-3.1-8b-instant", api_key=groq_api, streaming=True)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1471f82b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1471f8c70>, model_name='llama-3.1-8b-instant', groq_api_key=SecretStr('**********'), streaming=True)

In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



# loader = PyPDFLoader("apjspeech.pdf")
# docs = loader.load_and_split()
# docs
docs = PyPDFLoader("/Users/sachinsen/Documents/Generative Ai/Claases/18118 LEASE001.pdf").load_and_split()

In [17]:
docs

[Document(metadata={'source': '/Users/sachinsen/Documents/Generative Ai/Claases/18118 LEASE001.pdf', 'page': 0}, page_content='t\n@lndian-l,lon Judicial Stamp\nHaryana Govemmentt\n@ Dale:141132023\nI101Ce.tificale No.\nGRN No.G0a{202rc695\n1002 78$rne Duty P.id\nFadty: t0\nNanc :\nH.No/Floor\nCityMlhga\nPhona :Vsd(r LE\nN.\nGrruffl\n98"\'"28S.ei\\rvrd : t{3\ntxri, : Guu$a|rrl-Ildnat : lla\nSh|!: Hdya|a\nffi\nPurpose : AGREEMENT lo ba rub,nin d at Co[c.mad onlcrt&ll&\nnE rlttrtry ol tB d@rrurr cr! !. y.ra.d !y lcrEE !l.3 OC.d. Iko.Jgn !6rr prD.. s cr ib r.a.r!. nto.r,bgrrtry rEn\nLEASE DEED\nThis LEASE-DEED (hereafter referred to as "Lease Deed") is executed at Gurugram on this\n- day of\nBY\nVatika Ltd. a Company incorporated underthe Companies Act 1956 having its registered office at\nUnit No.-A-002, INXT City Centre, Ground Floor, Tower A, Sector 83, vatika lndia Next\nGurgaon - 1220\'12 acting through its duly authorized signatory Ms. Ritu Gupta, (hereinafter\nReferred to as "Lessor

In [18]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'source': '/Users/sachinsen/Documents/Generative Ai/Claases/18118 LEASE001.pdf', 'page': 0}, page_content='t\n@lndian-l,lon Judicial Stamp\nHaryana Govemmentt\n@ Dale:141132023\nI101Ce.tificale No.\nGRN No.G0a{202rc695\n1002 78$rne Duty P.id\nFadty: t0\nNanc :\nH.No/Floor\nCityMlhga\nPhona :Vsd(r LE\nN.\nGrruffl\n98"\'"28S.ei\\rvrd : t{3\ntxri, : Guu$a|rrl-Ildnat : lla\nSh|!: Hdya|a\nffi\nPurpose : AGREEMENT lo ba rub,nin d at Co[c.mad onlcrt&ll&\nnE rlttrtry ol tB d@rrurr cr! !. y.ra.d !y lcrEE !l.3 OC.d. Iko.Jgn !6rr prD.. s cr ib r.a.r!. nto.r,bgrrtry rEn\nLEASE DEED\nThis LEASE-DEED (hereafter referred to as "Lease Deed") is executed at Gurugram on this\n- day of\nBY\nVatika Ltd. a Company incorporated underthe Companies Act 1956 having its registered office at\nUnit No.-A-002, INXT City Centre, Ground Floor, Tower A, Sector 83, vatika lndia Next\nGurgaon - 1220\'12 acting through its duly authorized signatory Ms. Ritu Gupta, (hereinafter\nReferred to as "Lessor

In [19]:
len(final_documents)

54

In [26]:
chunks_prompt="""
Please summarize the below documents and take data in pointers:
context:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [27]:
final_prompt='''
Provide Final summary in single string in below points and provide in pointers as well
1. Lessee name
2. Lessee name
3. Monthly Rent in INR
4. Rental Value 
5. Escalation
6. Notice Period
7. leased property address

context:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], template='\nProvide Final summary in single string in below points and provide in pointers as well\n1. Lessee name\n2. Lessee name\n3. Monthly Rent in INR\n4. Rental Value \n5. Escalation\n6. Notice Period\n7. leased property address\n\ncontext:{text}\n\n')

In [28]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below documents and take data in pointers:
context:`t
@lndian-l,lon Judicial Stamp
Haryana Govemmentt
@ Dale:141132023
I101Ce.tificale No.
GRN No.G0a{202rc695
1002 78$rne Duty P.id
Fadty: t0
Nanc :
H.No/Floor
CityMlhga
Phona :Vsd(r LE
N.
Grruffl
98"'"28S.ei\rvrd : t{3
txri, : Guu$a|rrl-Ildnat : lla
Sh|!: Hdya|a
ffi
Purpose : AGREEMENT lo ba rub,nin d at Co[c.mad onlcrt&ll&
nE rlttrtry ol tB d@rrurr cr! !. y.ra.d !y lcrEE !l.3 OC.d. Iko.Jgn !6rr prD.. s cr ib r.a.r!. nto.r,bgrrtry rEn
LEASE DEED
This LEASE-DEED (hereafter referred to as "Lease Deed") is executed at Gurugram on this
- day of
BY
Vatika Ltd. a Company incorporated underthe Companies Act 1956 having its registered office at
Unit No.-A-002, INXT City Centre, Ground Floor, Tower A, Sector 83, vatika lndia Next
Gurgaon - 1220'12 acting through its duly authorized signatory Ms. Ritu Gupta, (hereina

KeyboardInterrupt: 

In [ ]:
print(output)

Here's a final summary in a single string and with pointers for each of the provided points:

**Summary:**

"The commercial rental agreement outlines various guidelines and agreements including regulations for working hours, high-noise works, material restrictions, rent and GST, escalation, security deposit, notice period, and lock-in period. The key points of the agreement are the Lessee's right to peacefully hold and enjoy the Demised Premises, the Lessor's obligation to maintain the premises, the Lessee's obligation to pay rent and perform all terms and conditions, and the notice period for termination. The Lessee is liable for the entire rent and maintenance charges if they terminate the lease before the end of the lock-in period, and the Lessor can terminate the lease for non-performance of the Lessee's covenants or failure to rectify within the agreed time period."

**Pointers:**

1. **Key Parties:**
    - **Lessor (Landlord)**: The person or entity that owns the property being l

In [24]:
summary_chain.invoke(final_documents)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below documents and take data in pointers:
context:`t
@lndian-l,lon Judicial Stamp
Haryana Govemmentt
@ Dale:141132023
I101Ce.tificale No.
GRN No.G0a{202rc695
1002 78$rne Duty P.id
Fadty: t0
Nanc :
H.No/Floor
CityMlhga
Phona :Vsd(r LE
N.
Grruffl
98"'"28S.ei\rvrd : t{3
txri, : Guu$a|rrl-Ildnat : lla
Sh|!: Hdya|a
ffi
Purpose : AGREEMENT lo ba rub,nin d at Co[c.mad onlcrt&ll&
nE rlttrtry ol tB d@rrurr cr! !. y.ra.d !y lcrEE !l.3 OC.d. Iko.Jgn !6rr prD.. s cr ib r.a.r!. nto.r,bgrrtry rEn
LEASE DEED
This LEASE-DEED (hereafter referred to as "Lease Deed") is executed at Gurugram on this
- day of
BY
Vatika Ltd. a Company incorporated underthe Companies Act 1956 having its registered office at
Unit No.-A-002, INXT City Centre, Ground Floor, Tower A, Sector 83, vatika lndia Next
Gurgaon - 1220'12 acting through its duly authorized signatory Ms. Ritu Gupta, (hereina

KeyboardInterrupt: 

In [ ]:
text = """'output_text': '**Summary:**\n\nThe document outlines a commercial lease agreement between Vatika Ltd. (Lessor) and Gogoro India Private Limited (Lessee) for the lease of a property in Sector 43, Gurugram. Key terms include:\n\n- Monthly rent of Rs. 55/sq. ft., escalating to Rs. 65 and 75 in subsequent years.\n- 3-year lease term with a 3-month lock-in period.\n- 15% escalation clause after the first year.\n- Shared property tax responsibility.\n- Refundable fit-out deposit and damage responsibility outlined.\n\n**Pointers:**\n\n- Lessee responsibilities include rent payment, property maintenance, and repairs.\n- Lessor responsibilities include building structure maintenance and insurance coverage.\n- Specific rules regarding access, alterations, and security are outlined.\n- Confidentiality and signature details are mentioned.\n- Fit-out guidelines and payment terms are summarized.\n- Billing information including contact details and payment mode are provided.'}"""

In [ ]:
print(text)